In [35]:
from pyspark.sql.functions import *

custom_attribute_parent_folder = "s3://experian-jamloop-us-east-1-1872955632/experian/20251107/custom_audiences/JamloopCompany/"
file_name_datepart="20251107140237"
file_name_customize="ROBERTSTOYOTA_NOV20251106_994380"

aaid_attribute_files = f"AAID_{file_name_datepart}_JamloopCompany_JAMLOOP_{file_name_customize}_*_*_*.csv.gz"
idfa_attribute_files = f"IDFA_{file_name_datepart}_JamloopCompany_JAMLOOP_{file_name_customize}_*_*_*.csv.gz"
ip_attriute_files = f"IP_{file_name_datepart}_JamloopCompany_JAMLOOP_{file_name_customize}_*_*_*.csv.gz"
luid_attribute_files = f"LUID_{file_name_datepart}_JamloopCompany_JAMLOOP_{file_name_customize}_*_*_*.csv.gz"

output_path = "s3a://experian-jamloop-us-east-1-1872955632/jamloop_output/2511070013"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
#df_aaid = spark.read.option('delimiter','|').csv(custom_attribute_parent_folder + aaid_attribute_files, header=False)
#df_aaid.show(truncate=False)

#df_luid = spark.read.option('delimiter','|').csv(custom_attribute_parent_folder + luid_attribute_files, header=False)
#df_luid.show(truncate = False)


#df_idfa = spark.read.option('delimiter','|').csv(custom_attribute_parent_folder + idfa_attribute_files, header=False)
#df_idfa.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
df_ip = spark.read.option('delimiter','|').csv(custom_attribute_parent_folder + ip_attriute_files, header=False)

# Show the results
df_ip.show(truncate = False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------+----------+
|_c0                  |_c1       |
+---------------------+----------+
|2607:fb90:234f:16da::|2511070013|
|98.36.97.160         |2511070013|
|75.245.181.212       |2511070013|
|2600:1702:46a8:2290::|2511070013|
|2600:1003:b01e:e7ce::|2511070013|
|2600:1004:b146:c84c::|2511070013|
|2600:1005:b040:876c::|2511070013|
|2600:6c58:2a00:29::  |2511070013|
|2600:1008:b324:e7e0::|2511070013|
|24.111.91.168        |2511070013|
|172.82.146.129       |2511070013|
|99.14.54.58          |2511070013|
|69.137.75.79         |2511070013|
|2600:1700:2660:2850::|2511070013|
|2600:100b:b11d:ccf1::|2511070013|
|2600:1005:b1c7:cf70::|2511070013|
|2603:9001:3700:4c08::|2511070013|
|24.40.250.129        |2511070013|
|2600:1004:b332:9698::|2511070013|
|98.34.42.165         |2511070013|
+---------------------+----------+
only showing top 20 rows

In [38]:
# run this function if you need replace segment ID with Jamloop own
df_ip_old = spark.read.option('delimiter','|').csv(custom_attribute_parent_folder + ip_attriute_files, header=False)
# Create a new DataFrame with replaced values in the second column
df_ip = df_ip_old.withColumn('_c1', 
        regexp_replace(
            col('_c1'),
            '2506170085',
            '10719'
        )
)

# Show the results
df_ip.show(truncate = False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------+----------+
|_c0                  |_c1       |
+---------------------+----------+
|2607:fb90:234f:16da::|2511070013|
|98.36.97.160         |2511070013|
|75.245.181.212       |2511070013|
|2600:1702:46a8:2290::|2511070013|
|2600:1003:b01e:e7ce::|2511070013|
|2600:1004:b146:c84c::|2511070013|
|2600:1005:b040:876c::|2511070013|
|2600:6c58:2a00:29::  |2511070013|
|2600:1008:b324:e7e0::|2511070013|
|24.111.91.168        |2511070013|
|172.82.146.129       |2511070013|
|99.14.54.58          |2511070013|
|69.137.75.79         |2511070013|
|2600:1700:2660:2850::|2511070013|
|2600:100b:b11d:ccf1::|2511070013|
|2600:1005:b1c7:cf70::|2511070013|
|2603:9001:3700:4c08::|2511070013|
|24.40.250.129        |2511070013|
|2600:1004:b332:9698::|2511070013|
|98.34.42.165         |2511070013|
+---------------------+----------+
only showing top 20 rows

In [39]:
# add prefix
vendor_prefix = "EXP-"

def process_and_print_non_integer_ids(line):
    #ip, ids_str = line.split(maxsplit=1)
    ip = line[0]
    ids = line[1]
    processed_ids = []
    for id in ids.split(','):
        print(id)
        processed_ids.append(vendor_prefix + id)

    return ('+'+ip, processed_ids)

rdd_result = df_ip.rdd.map(process_and_print_non_integer_ids)
rdd_result.take(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('+2607:fb90:234f:16da::', ['EXP-2511070013']), ('+98.36.97.160', ['EXP-2511070013']), ('+75.245.181.212', ['EXP-2511070013']), ('+2600:1702:46a8:2290::', ['EXP-2511070013']), ('+2600:1003:b01e:e7ce::', ['EXP-2511070013'])]

In [40]:
rdd_output = rdd_result.map(lambda x: f"{x[0]} {','.join(x[1])}")
rdd_output.take(20)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['+2607:fb90:234f:16da:: EXP-2511070013', '+98.36.97.160 EXP-2511070013', '+75.245.181.212 EXP-2511070013', '+2600:1702:46a8:2290:: EXP-2511070013', '+2600:1003:b01e:e7ce:: EXP-2511070013', '+2600:1004:b146:c84c:: EXP-2511070013', '+2600:1005:b040:876c:: EXP-2511070013', '+2600:6c58:2a00:29:: EXP-2511070013', '+2600:1008:b324:e7e0:: EXP-2511070013', '+24.111.91.168 EXP-2511070013', '+172.82.146.129 EXP-2511070013', '+99.14.54.58 EXP-2511070013', '+69.137.75.79 EXP-2511070013', '+2600:1700:2660:2850:: EXP-2511070013', '+2600:100b:b11d:ccf1:: EXP-2511070013', '+2600:1005:b1c7:cf70:: EXP-2511070013', '+2603:9001:3700:4c08:: EXP-2511070013', '+24.40.250.129 EXP-2511070013', '+2600:1004:b332:9698:: EXP-2511070013', '+98.34.42.165 EXP-2511070013']

In [41]:
# Repartition the RDD into 256 parts
rdd_repartitioned = rdd_output.repartition(256)

# Set compression codec to Gzip and save the consolidated result back to S3
spark.sparkContext._jsc.hadoopConfiguration().set("mapred.output.compress", "true")
spark.sparkContext._jsc.hadoopConfiguration().set("mapred.output.compression.codec", "org.apache.hadoop.io.compress.GzipCodec")

# Save the consolidated result back to S3
rdd_repartitioned.saveAsTextFile(output_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
# Check the output
df = spark.read.text(output_path + "/*.gz")
#df.count()
df.show(20, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------------------------+
|value                                |
+-------------------------------------+
|+174.118.120.241 EXP-2511070013      |
|+2603:6080:1900:a207:: EXP-2511070013|
|+2600:100f:b12c:b0de:: EXP-2511070013|
|+2600:1009:b1a8:ad2d:: EXP-2511070013|
|+2600:1005:b145:1d82:: EXP-2511070013|
|+47.204.139.8 EXP-2511070013         |
|+76.158.235.223 EXP-2511070013       |
|+76.248.239.120 EXP-2511070013       |
|+2607:fb91:50b:805:: EXP-2511070013  |
|+2600:382:96f4:ab:: EXP-2511070013   |
|+71.68.128.85 EXP-2511070013         |
|+2607:fb91:2f8c:c242:: EXP-2511070013|
|+2607:fb91:2630:c197:: EXP-2511070013|
|+2600:1010:b1f6:8187:: EXP-2511070013|
|+99.125.117.229 EXP-2511070013       |
|+68.53.0.184 EXP-2511070013          |
|+66.56.236.102 EXP-2511070013        |
|+2607:fb90:45e1:5108:: EXP-2511070013|
|+2607:fb91:3238:4523:: EXP-2511070013|
|+2a04:4e41:3fe8:ab7d:: EXP-2511070013|
+-------------------------------------+
only showing top 20 rows